This notebook contains the commands to pre-process the files to be used in the MR analysis using blood eQTL from different cohorts.

In [ ]:
#Information to provide:
pathMR #Path to run the MR analyses
pathdata #Path with the datasets
setwd(pathMR)
path_GALA #Path to the summary statistics from the GALA study
path_eQTLGen #Path to the summary statistics from the eQTLGen study

In [ ]:
library(data.table)

# Get the data (only once)

# eQTL

## GENOA

In [ ]:
setwd(pathdata)
system("wget http://www.xzlab.org/data/AA_summary_statistics.txt.gz")
system("mv AA_summary_statistics.txt.gz GENOA_AA_summary_statistics.txt.gz")
system("wget http://www.xzlab.org/data/EA_summary_statistics.txt.gz")
system("mv EA_summary_statistics.txt.gz GENOA_EA_summary_statistics.txt.gz")

In this dataset, the effect allele is not clearly provided, but only 'allele1' which corresponds to the minor allele, and 'allele0' which corresponds to the major allele (https://xiangzhou.github.io/resources/GENOA_eQTL_README.txt). There is no example in the paper which can help to determine which is the effect allele. We will therefore here assume that the effect allele is allele1, the minor allele.

#### Transform the data in the right format

In [ ]:
GENOA_EA <- fread(paste0(pathdata, "GENOA_AA_summary_statistics.txt.gz"), header = T)
head(GENOA_EA)
dim(GENOA_EA)
GENOA_AA <- fread(paste0(pathdata, "GENOA_EA_summary_statistics.txt.gz"), header = T)
head(GENOA_AA)
dim(GENOA_AA)

In [ ]:
colnames(GENOA_EA) <- colnames(GENOA_AA) <- c("chr", "mol.trait", "rsid", "pos", "ea", "nea", "eaf", "beta", "se", "pval")
GENOA_EA$qtl.GW <- ifelse(GENOA_EA$pval<5e-8, T, F)
GENOA_AA$qtl.GW <- ifelse(GENOA_AA$pval<5e-8, T, F)
GENOA_EA$qtl.study <- ifelse(GENOA_EA$pval< 1.385504e-4, T, F)
GENOA_AA$qtl.study <- ifelse(GENOA_AA$pval< 6.245907e-5, T, F)
GENOA_EA$n <- 801 ; GENOA_AA$n <- 1032
GENOA_EA$ID <- paste0(GENOA_EA$chr, ":", GENOA_EA$pos, "_", GENOA_EA$ea, "_", GENOA_EA$nea, "_37")
GENOA_AA$ID <- paste0(GENOA_AA$chr, ":", GENOA_AA$pos, "_", GENOA_AA$ea, "_", GENOA_AA$nea, "_37")

In [ ]:
#Export the data
data.table::fwrite(GENOA_EA, file = paste0(pathdata, "eQTL_GENOA_plasma_EA_801_37.txt.gz"), sep = "\t")
data.table::fwrite(GENOA_AA, file = paste0(pathdata, "eQTL_GENOA_plasma_AA_1032_37.txt.gz"), sep = "\t")

## GALAII and SAGE
Look at readme_GALAII for more informations on file format

In [ ]:
pop = "AA" #Population to consider with the GALAII study

In [ ]:
#Import all the files, only keeping the significant QTL (column 12 from the file)
allchr.files <- do.call(rbind, lapply(1:22, function(z) fread(cmd = paste0("zcat ", path_GALA, pop, "/", z, ".allpairs.more.galasage.hg38.txt.gz | awk '{if($12 == \"TRUE\") {print $0} }'" ))))

In [ ]:
colnames(allchr.files) <- c("gene_id", "variant_id", "tss_distance", "ma_samples", "ma_count", "maf", "pval_nominal", "slope", "slope_se", "qval", "pval_nominal_threshold", "eQTL.flag", "vid2")
head(allchr.files)

gene_id,variant_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,qval,pval_nominal_threshold,eQTL.flag,vid2
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>
ENSG00000000457.14,rs74898833,111830,109,113,0.0746367,4.17900e-16,0.624650,0.0749591,4.02204e-69,0.000467711,TRUE,chr1_170006097_A_G_b38
ENSG00000000457.14,rs4656729,108084,554,739,0.4881110,3.67768e-19,0.356575,0.0387043,4.02204e-69,0.000467711,TRUE,chr1_170002351_T_C_b38
ENSG00000000457.14,rs61825344,111776,101,105,0.0693527,2.78993e-05,0.338343,0.0802136,4.02204e-69,0.000467711,TRUE,chr1_170006043_A_C_b38
ENSG00000000457.14,rs61825341,110233,101,105,0.0693527,2.78993e-05,0.338343,0.0802136,4.02204e-69,0.000467711,TRUE,chr1_170004500_A_G_b38
ENSG00000000457.14,rs151032917,111014,101,105,0.0693527,2.78993e-05,0.338343,0.0802136,4.02204e-69,0.000467711,TRUE,chr1_170005281_T_C_b38
ENSG00000000457.14,rs2474708,112955,250,273,0.1803170,8.76814e-11,0.347814,0.0527922,4.02204e-69,0.000467711,TRUE,chr1_170007222_C_T_b38


In [ ]:
#Extract chr, pos, ref and alt
allchr.files <- allchr.files[, c("chr", "pos", "ref", "alt", "build") := tstrsplit(vid2, "_", fixed=TRUE)]
allchr.files$chr <- gsub(allchr.files$chr, pattern = "chr", replacement = "")

In [ ]:
head(allchr.files)
range(allchr.files$maf)
sum(allchr.files$ma_count > (757*2)) #We here consider that the alt allele is the minor allele

gene_id,variant_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,qval,pval_nominal_threshold,eQTL.flag,vid2,chr,pos,ref,alt,build
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ENSG00000000457.14,rs74898833,111830,109,113,0.0746367,4.17900e-16,0.624650,0.0749591,4.02204e-69,0.000467711,TRUE,chr1_170006097_A_G_b38,1,170006097,A,G,b38
ENSG00000000457.14,rs4656729,108084,554,739,0.4881110,3.67768e-19,0.356575,0.0387043,4.02204e-69,0.000467711,TRUE,chr1_170002351_T_C_b38,1,170002351,T,C,b38
ENSG00000000457.14,rs61825344,111776,101,105,0.0693527,2.78993e-05,0.338343,0.0802136,4.02204e-69,0.000467711,TRUE,chr1_170006043_A_C_b38,1,170006043,A,C,b38
ENSG00000000457.14,rs61825341,110233,101,105,0.0693527,2.78993e-05,0.338343,0.0802136,4.02204e-69,0.000467711,TRUE,chr1_170004500_A_G_b38,1,170004500,A,G,b38
ENSG00000000457.14,rs151032917,111014,101,105,0.0693527,2.78993e-05,0.338343,0.0802136,4.02204e-69,0.000467711,TRUE,chr1_170005281_T_C_b38,1,170005281,T,C,b38
ENSG00000000457.14,rs2474708,112955,250,273,0.1803170,8.76814e-11,0.347814,0.0527922,4.02204e-69,0.000467711,TRUE,chr1_170007222_C_T_b38,1,170007222,C,T,b38


[1] 0.010568 0.500000

[1] 0

In [ ]:
#Only keeping the significant eQTL
allchr.files.sig <- subset(allchr.files, eQTL.flag)
dim(allchr.files)
dim(allchr.files.sig)

[1] 4282985      18

[1] 4282985      18

In [ ]:
#Export the file
if(pop=="AA"){prefix.file = "eQTL_SAGE_"}else{prefix.file = "eQTL_GALAII_"}
system(paste0("mkdir -p ", pathdata, prefix.file, pop))
if(!file.exists(paste0(pathdata, prefix.file, pop, "/allchr.pairseQTLTRUE.more.galasage.", pop, ".hg38.txt.gz"))){
    fwrite(allchr.files.sig, file = paste0(pathdata, prefix.file, pop, "/allchr.pairseQTLTRUE.more.galasage.", pop, ".hg38.txt.gz"), sep = "\t")
}
#Export the whole file for PWCoCo
if(!file.exists(paste0(pathdata, prefix.file, pop, "/allchr.allcisQTL.more.galasage.", pop, ".hg38.txt.gz"))){
    fwrite(allchr.files, file = paste0(pathdata, prefix.file, pop, "/allchr.allcisQTL.more.galasage.", pop, ".hg38.txt.gz"), sep = "\t")
}

# eQTLGen

In [ ]:
#Import file
data.eQTLGen <- fread(paste0(path_eQTLGen, "eQTLGen_2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt.gz"))
head(data.eQTLGen)

Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP
<dbl>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
3.2717e-310,rs12230244,12,10117369,T,A,200.7534,ENSG00000172322,CLEC12A,12,10126104,34,30596,0,4.1662e-302
3.2717e-310,rs12229020,12,10117683,G,C,200.6568,ENSG00000172322,CLEC12A,12,10126104,34,30596,0,4.1662e-302
3.2717e-310,rs61913527,12,10116198,T,C,200.2654,ENSG00000172322,CLEC12A,12,10126104,34,30598,0,4.1662e-302
3.2717e-310,rs2594103,12,10115428,T,C,200.0420,ENSG00000172322,CLEC12A,12,10126104,34,30598,0,4.1662e-302
3.2717e-310,rs12231833,12,10118428,A,G,199.9508,ENSG00000172322,CLEC12A,12,10126104,34,30592,0,4.1662e-302
3.2717e-310,rs12231872,12,10118747,C,G,199.7708,ENSG00000172322,CLEC12A,12,10126104,33,30268,0,4.1662e-302


# eQTL GTEx

In [ ]:
# Import the eQTL data for all significant cis pairs (has been downloaded from https://www.gtexportal.org/home/downloads/adult-gtex/qtl)
GTEx.cis.blood.sig <- as.data.frame(fread("/lustre/groups/itg/teams/zeggini/projects/T2D-diamante/T2DGGI_MR/datasets/GTEx/GTEx_Analysis_v8_eQTL/Whole_Blood.v8.signif_variant_gene_pairs.txt.gz"))
head(GTEx.cis.blood.sig)

,variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,pval_nominal_threshold,min_pval_nominal,pval_beta
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1_64764_C_T_b38,ENSG00000227232.5,35211,79,80,0.0597015,1.72756e-09,0.516614,0.0844652,0.000393213,3.85133e-13,1.05258e-09
2,chr1_665098_G_A_b38,ENSG00000227232.5,635545,140,146,0.1089550,8.84898e-13,0.462697,0.0633368,0.000393213,3.85133e-13,1.05258e-09
3,chr1_666028_G_A_b38,ENSG00000227232.5,636475,125,131,0.0977612,3.85133e-13,0.489662,0.0659359,0.000393213,3.85133e-13,1.05258e-09
4,chr1_17556_C_T_b38,ENSG00000238009.6,-111667,27,27,0.0201493,1.48531e-06,0.760661,0.1564470,0.000333231,4.99434e-31,1.05973e-25
5,chr1_58814_G_A_b38,ENSG00000238009.6,-70409,109,119,0.0888060,1.49434e-18,0.636831,0.0700984,0.000333231,4.99434e-31,1.05973e-25
6,chr1_60351_A_G_b38,ENSG00000238009.6,-68872,83,92,0.0686567,6.43685e-13,0.575975,0.0783446,0.000333231,4.99434e-31,1.05973e-25


In [ ]:
# Extract chr pos ref alt
chrposrefalt <- lapply(GTEx.cis.blood.sig[,"variant_id"], function(z) strsplit(z, split = "_"))

In [ ]:
GTEx.cis.blood.sig$chr <- unlist(lapply(chrposrefalt, function(z) gsub(z[[1]][1], pattern = "chr", replacement = "")))
GTEx.cis.blood.sig$pos <- unlist(lapply(chrposrefalt, function(z) z[[1]][2]))
GTEx.cis.blood.sig$ref <- unlist(lapply(chrposrefalt, function(z) z[[1]][3]))
GTEx.cis.blood.sig$alt <- unlist(lapply(chrposrefalt, function(z) z[[1]][4]))

In [ ]:
head(GTEx.cis.blood.sig)
range(GTEx.cis.blood.sig$maf)
sum(GTEx.cis.blood.sig$ma_count > (620*2))
#Here we consider than maf is frequency of alternative allele

,variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,pval_nominal_threshold,min_pval_nominal,pval_beta,chr,pos,ref,alt
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,chr1_64764_C_T_b38,ENSG00000227232.5,35211,79,80,0.0597015,1.72756e-09,0.516614,0.0844652,0.000393213,3.85133e-13,1.05258e-09,1,64764,C,T
2,chr1_665098_G_A_b38,ENSG00000227232.5,635545,140,146,0.1089550,8.84898e-13,0.462697,0.0633368,0.000393213,3.85133e-13,1.05258e-09,1,665098,G,A
3,chr1_666028_G_A_b38,ENSG00000227232.5,636475,125,131,0.0977612,3.85133e-13,0.489662,0.0659359,0.000393213,3.85133e-13,1.05258e-09,1,666028,G,A
4,chr1_17556_C_T_b38,ENSG00000238009.6,-111667,27,27,0.0201493,1.48531e-06,0.760661,0.1564470,0.000333231,4.99434e-31,1.05973e-25,1,17556,C,T
5,chr1_58814_G_A_b38,ENSG00000238009.6,-70409,109,119,0.0888060,1.49434e-18,0.636831,0.0700984,0.000333231,4.99434e-31,1.05973e-25,1,58814,G,A
6,chr1_60351_A_G_b38,ENSG00000238009.6,-68872,83,92,0.0686567,6.43685e-13,0.575975,0.0783446,0.000333231,4.99434e-31,1.05973e-25,1,60351,A,G


[1] 0.00447761 0.50000000

[1] 0

In [ ]:
#Export the file
system(paste0("mkdir -p ", pathdata, "eQTL_GTEx_EUR/"))
if(!file.exists(paste0(pathdata, "/eQTL_GTEx_EUR/Whole_Blood.v8.signif_variant_gene_pairs.txt.gz"))){
    fwrite(GTEx.cis.blood.sig, file = paste0(pathdata, "/eQTL_GTEx_EUR/Whole_Blood.v8.signif_variant_gene_pairs_chrpos.txt.gz"), sep = "\t")
}

## eQTLGen

In [ ]:
#Import information on the variants: frequency
freq.eQTLGen <- as.data.frame(fread(paste0(path_eQTLGen, "cis-eQTLs-full_eQTLGen_AF_incl_nr_formatted_20191212.new.txt_besd-dense.esi.gz")))
#From a few check on the file, it seems that allele in V5 is the allele with the frequency
#Also seems to correspond to the AssessedAllele in the other file
colnames(freq.eQTLGen) <- c("SNPChr", "SNP", "V3", "SNPPos", "AssessedAllele", "OtherAllele", "Freq")
head(freq.eQTLGen)
#Import information on significant cisQTL
sig.cis.eQTLGen <- as.data.frame(fread(paste0(path_eQTLGen, "2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt.gz")))
head(sig.cis.eQTLGen)

,SNPChr,SNP,V3,SNPPos,AssessedAllele,OtherAllele,Freq
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>
1,1,rs181431124,0,64649,C,A,0.0111492
2,1,rs147502335,0,135203,A,G,0.0237430
3,1,rs147589465,0,240789,G,T,0.0117032
4,1,rs145971835,0,534545,A,G,0.1064190
5,1,rs148329687,0,568709,G,A,0.0636833
6,1,rs140832136,0,603505,C,A,0.0227799


,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP
,<dbl>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,3.2717e-310,rs12230244,12,10117369,T,A,200.7534,ENSG00000172322,CLEC12A,12,10126104,34,30596,0,4.1662e-302
2,3.2717e-310,rs12229020,12,10117683,G,C,200.6568,ENSG00000172322,CLEC12A,12,10126104,34,30596,0,4.1662e-302
3,3.2717e-310,rs61913527,12,10116198,T,C,200.2654,ENSG00000172322,CLEC12A,12,10126104,34,30598,0,4.1662e-302
4,3.2717e-310,rs2594103,12,10115428,T,C,200.0420,ENSG00000172322,CLEC12A,12,10126104,34,30598,0,4.1662e-302
5,3.2717e-310,rs12231833,12,10118428,A,G,199.9508,ENSG00000172322,CLEC12A,12,10126104,34,30592,0,4.1662e-302
6,3.2717e-310,rs12231872,12,10118747,C,G,199.7708,ENSG00000172322,CLEC12A,12,10126104,33,30268,0,4.1662e-302


In [ ]:
#Combine the information
dim(freq.eQTLGen)
dim(sig.cis.eQTLGen)
sig.cis.eQTLGen.freq <- merge(sig.cis.eQTLGen, freq.eQTLGen, by = c("SNPChr", "SNPPos", "SNP", "AssessedAllele", "OtherAllele"))
dim(sig.cis.eQTLGen.freq)
head(sig.cis.eQTLGen.freq)

[1] 8932843       7

[1] 10507664       15

[1] 10507664       17

,SNPChr,SNPPos,SNP,AssessedAllele,OtherAllele,Pvalue,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP,V3,Freq
,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>
1,1,100000012,rs10875231,T,G,3.0712e-06,-4.6660,ENSG00000079335,CDC14A,1,100898208,36,31644,0.008209723,1.000000e+00,0,0.2681990
2,1,100003419,rs114427610,T,C,1.1180e-05,4.3931,ENSG00000122477,LRRC39,1,100629090,28,23230,0.028987748,1.000000e+00,0,0.0321567
3,1,10000400,rs1237370,T,A,1.1276e-64,-16.9814,ENSG00000162444,RBP7,1,10066671,34,31142,0.000000000,1.435900e-56,0,0.1564430
4,1,10000400,rs1237370,T,A,1.7605e-17,-8.5085,ENSG00000130939,UBE4B,1,10167093,34,31142,0.000000000,2.241900e-09,0,0.1564430
5,1,10000400,rs1237370,T,A,2.0627e-10,6.3567,ENSG00000178585,CTNNBIP1,1,9939364,32,30604,0.000000000,2.626679e-02,0,0.1564430
6,1,10000400,rs1237370,T,A,2.0923e-18,-8.7521,ENSG00000054523,KIF1B,1,10356262,34,31142,0.000000000,2.664400e-10,0,0.1564430


In [ ]:
#Compute the beta based on Z-score and frequency using information in README file from https://www.eqtlgen.org/cis-eqtls.html
sig.cis.eQTLGen.freq$beta <- sig.cis.eQTLGen.freq$Zscore / sqrt(2*sig.cis.eQTLGen.freq$Freq * (1-sig.cis.eQTLGen.freq$Freq) * (sig.cis.eQTLGen.freq$NrSamples + sig.cis.eQTLGen.freq$Zscore**2))
sig.cis.eQTLGen.freq$se <- 1 / sqrt(2*sig.cis.eQTLGen.freq$Freq * (1-sig.cis.eQTLGen.freq$Freq) * (sig.cis.eQTLGen.freq$NrSamples + sig.cis.eQTLGen.freq$Zscore**2))

In [ ]:
#Export the file
system(paste0("mkdir -p ", pathdata, "eQTL_eQTLGen_EUR"))
if(!file.exists(paste0(pathdata, "eQTL_eQTLGen_EUR/2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.EUR.txt.gz"))){
    fwrite(sig.cis.eQTLGen.freq, file = paste0(pathdata, "eQTL_eQTLGen_EUR/2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.EUR.txt.gz"), sep = "\t")
}